In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from datetime import datetime

In [2]:
# loading announcements
mazda=pd.read_json('dataset_mazda.json')

### text cleaning to gain classification (dictionary values)

In [8]:
# the column "lokalizacja" contains the description of the location, but it is different for each record
mazda[['lokalizacja']].head() 

,lokalizacja
0,"Piła, pilski, Wielkopolskie"
1,"Kwidzyn, kwidzyński, Pomorskie"
10,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ..."
100,"Gdańsk, Pomorskie, Przymorze Małe"
1000,"Estrady 14 - 05-080 Klaudyn, warszawski zachod..."


###### preparing the values (list of regions)

In [9]:
# list
regions = ['Dolnoslaskie', 'Kujawsko-pomorskie', 'Lubelskie', 'Lubuskie', 'Lodzkie',
'Malopolskie', 'Mazowieckie', 'Opolskie', 'Podkarpackie', 'Podlaskie', 'Pomorskie',
'Slaskie', 'Swietokrzyskie', 'Warminsko-mazurskie', 'Wielkopolskie', 'Zachodniopomorskie']

In [10]:
# extra column
mazda = mazda.assign(lokalizacja_wojewodztwo = mazda['lokalizacja']) #.str.lower())

# text unification
mazda['lokalizacja_wojewodztwo'].replace({'ś':'s'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Ś':'S'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'ą':'a'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'ł':'l'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Ł':'L'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'ó':'o'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'ę':'e'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'ń':'n'},regex=True, inplace=True)

In [11]:
mazda[['lokalizacja','lokalizacja_wojewodztwo']].head()

,lokalizacja,lokalizacja_wojewodztwo
0,"Piła, pilski, Wielkopolskie","Pila, pilski, Wielkopolskie"
1,"Kwidzyn, kwidzyński, Pomorskie","Kwidzyn, kwidzynski, Pomorskie"
10,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...","Aleja Krakowska 169 - 02-180 Warszawa, Wlochy ..."
100,"Gdańsk, Pomorskie, Przymorze Małe","Gdansk, Pomorskie, Przymorze Male"
1000,"Estrady 14 - 05-080 Klaudyn, warszawski zachod...","Estrady 14 - 05-080 Klaudyn, warszawski zachod..."


###### assigning uniform values

In [12]:
# searching for a value from the list in the description
for val in mazda['lokalizacja_wojewodztwo']:
    for reg in regions:
        if reg in val:
            #mazda['lokalizacja_wojewodztwo'].loc[( mazda['lokalizacja_wojewodztwo'] == val)] = reg # also work
            mazda['lokalizacja_wojewodztwo'].replace(val,reg, inplace=True)

In [13]:
 mazda[['lokalizacja','lokalizacja_wojewodztwo']].head()

,lokalizacja,lokalizacja_wojewodztwo
0,"Piła, pilski, Wielkopolskie",Wielkopolskie
1,"Kwidzyn, kwidzyński, Pomorskie",Pomorskie
10,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...","Aleja Krakowska 169 - 02-180 Warszawa, Wlochy ..."
100,"Gdańsk, Pomorskie, Przymorze Małe",Pomorskie
1000,"Estrady 14 - 05-080 Klaudyn, warszawski zachod...",Mazowieckie


In [14]:
# marking values from outside the list as "inne"
mazda['lokalizacja_wojewodztwo'] = mazda['lokalizacja_wojewodztwo'].apply(lambda x: 'inne' if x not in regions else x)

In [15]:
 mazda[['lokalizacja','lokalizacja_wojewodztwo']].head()

,lokalizacja,lokalizacja_wojewodztwo
0,"Piła, pilski, Wielkopolskie",Wielkopolskie
1,"Kwidzyn, kwidzyński, Pomorskie",Pomorskie
10,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...",inne
100,"Gdańsk, Pomorskie, Przymorze Małe",Pomorskie
1000,"Estrady 14 - 05-080 Klaudyn, warszawski zachod...",Mazowieckie


In [16]:
 mazda['lokalizacja_wojewodztwo'].unique()

array(['Wielkopolskie', 'Pomorskie', 'inne', 'Mazowieckie',
       'Swietokrzyskie', 'Malopolskie', 'Lodzkie', 'Lubelskie', 'Slaskie',
       'Opolskie', 'Kujawsko-pomorskie', 'Warminsko-mazurskie',
       'Lubuskie', 'Dolnoslaskie', 'Podkarpackie', 'Zachodniopomorskie',
       'Podlaskie'], dtype=object)

In [17]:
# how many "inne" items do we have?
# over 1000 records has not name of region in description of location
mazda.groupby(by=['lokalizacja_wojewodztwo']).size().sort_values()

lokalizacja_wojewodztwo
Opolskie                 96
Warminsko-mazurskie     103
Podlaskie               124
Zachodniopomorskie      134
Swietokrzyskie          144
Lubuskie                164
Podkarpackie            170
Lubelskie               213
Pomorskie               240
Kujawsko-pomorskie      242
Lodzkie                 282
Dolnoslaskie            350
Malopolskie             354
Wielkopolskie           579
Slaskie                 627
Mazowieckie             750
inne                   1089
dtype: int64

In [18]:
mazda['lokalizacja'][( mazda['lokalizacja_wojewodztwo'] == 'inne')]

10      Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...
1003    ul. Górczewska 30 - 01-147 Warszawa, Wola (Pol...
1004      Mińska 41a - 54-610 Wrocław, Fabryczna (Polska)
1005    ul. Górczewska 30 - 01-147 Warszawa, Wola (Pol...
1006      Mińska 41a - 54-610 Wrocław, Fabryczna (Polska)
1009    Sw Michała  20 - 61-023 Poznań, Nowe Miasto (P...
1037    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1038    DŹBOWSKA  66 - 42-202 Częstochowa, Stradom (Po...
1040        Rosoła 55 - 02-786 Warszawa, Ursynów (Polska)
1044    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1045    Sw Michała  20 - 61-023 Poznań, Nowe Miasto (P...
1046      Mińska 41a - 54-610 Wrocław, Fabryczna (Polska)
1047      Mińska 41a - 54-610 Wrocław, Fabryczna (Polska)
1049    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1050    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1051    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1052    Gen. Stanisława Maczka 136 - 15-691 Białystok,...
1055    ul.Jas

###### preparing the values (list of cities)

In [19]:
# list
city = ['Warszawa', 'Wrocław', 'Poznań', 'Częstochowa', 'Gdynia',
'Białystok', 'Kraków', 'Szczecin', 'Katowice', 'Gdańsk', 'Łódź']

In [20]:
# reset index because I get some error on index nr 222
mazda = mazda.reset_index(drop=True)

In [21]:
# change "inne" for description of loaction
mazda['lokalizacja_wojewodztwo'][( mazda['lokalizacja_wojewodztwo'] == 'inne')] = mazda['lokalizacja'].apply(lambda x: x)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
mazda[['lokalizacja','lokalizacja_wojewodztwo']].head()

,lokalizacja,lokalizacja_wojewodztwo
0,"Piła, pilski, Wielkopolskie",Wielkopolskie
1,"Kwidzyn, kwidzyński, Pomorskie",Pomorskie
2,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...","Aleja Krakowska 169 - 02-180 Warszawa, Włochy ..."
3,"Gdańsk, Pomorskie, Przymorze Małe",Pomorskie
4,"Estrady 14 - 05-080 Klaudyn, warszawski zachod...",Mazowieckie


###### assigning uniform values second time

In [23]:
# searching for a value from the list in the description
for val in mazda['lokalizacja_wojewodztwo']:
    for ci in city:
        if ci in val:
            mazda['lokalizacja_wojewodztwo'].replace(val,ci, inplace=True)    

In [24]:
mazda[['lokalizacja','lokalizacja_wojewodztwo']].head()

,lokalizacja,lokalizacja_wojewodztwo
0,"Piła, pilski, Wielkopolskie",Wielkopolskie
1,"Kwidzyn, kwidzyński, Pomorskie",Pomorskie
2,"Aleja Krakowska 169 - 02-180 Warszawa, Włochy ...",Warszawa
3,"Gdańsk, Pomorskie, Przymorze Małe",Pomorskie
4,"Estrady 14 - 05-080 Klaudyn, warszawski zachod...",Mazowieckie


In [25]:
# change city for region
mazda['lokalizacja_wojewodztwo'].replace({'Warszawa':'Mazowieckie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Wrocław':'Dolnoslaskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Poznań':'Wielkopolskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Częstochowa':'Slaskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Gdynia':'Pomorskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Białystok':'Podlaskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Kraków':'Malopolskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Szczecin':'Zachodniopomorskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Katowice':'Slaskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Gdańsk':'Pomorskie'},regex=True, inplace=True)
mazda['lokalizacja_wojewodztwo'].replace({'Łódź':'Lodzkie'},regex=True, inplace=True)

In [26]:
# marking values from outside the list as "inne"
mazda['lokalizacja_wojewodztwo'] = mazda['lokalizacja_wojewodztwo'].apply(lambda x: 'inne' if x not in regions else x)

In [27]:
# only 62 records assigned as "inne"
mazda.groupby(by=['lokalizacja_wojewodztwo']).size().sort_values()

lokalizacja_wojewodztwo
inne                     62
Opolskie                 96
Warminsko-mazurskie     103
Swietokrzyskie          144
Lubuskie                164
Podkarpackie            170
Zachodniopomorskie      173
Lubelskie               213
Kujawsko-pomorskie      242
Podlaskie               249
Lodzkie                 320
Pomorskie               352
Malopolskie             401
Dolnoslaskie            410
Wielkopolskie           615
Slaskie                 760
Mazowieckie            1187
dtype: int64

In [28]:
mazda['lokalizacja'][( mazda['lokalizacja_wojewodztwo'] == 'inne')]

136     Staromiejska 33 - 43-190 Mikołów, Paniowy (Pol...
208     PADEREWSKIEGO 124 - 42-400 Zawiercie, Warty (P...
266      Portowa 2 - 44-100 Gliwice, Śródmieście (Polska)
457     Spółdzielcza 38 - 58-500 Jelenia Góra, Centrum...
458     ROLNICZA 33 - 42-400 Zawiercie, Marciszów (Pol...
503     ul. Jeziorska➠Zawadzka 3 - 43-180 Orzesze, Cen...
509     Gliwicka 135A - 43-190 Mikołów, Śmiłowice (Pol...
589     wróblewskiego 27 - 42-506 Będzin, Grodziec (Po...
770     White House Business Centre Wilderspool Busine...
996     ul. Kozielska 82 - 44-121 Gliwice, Śródmieście...
1161    Pszczyńska 107A - 44-100 Gliwice, Politechnika...
1298    SIEWIERSKA 197a - 42-400 Zawiercie, Kromołów (...
1426    Tarnogórska 55 - 44-100 Gliwice, Śródmieście (...
1433          Małobądz - 42-500 Będzin, Małobądz (Polska)
1496    Al. Niepodległości 635 - 81-855 Sopot, Centrum...
1701    Okopowa 1 ( wjazd od ul. Zgorzeleckiej ) - 58-...
1779    ZGODY 20 (PRZY UL GLIWICKIEJ) - 43-190 Mikołów...
1883    Wolnoś

In [ ]:
mazda.to_json(r'C:\...\dataset_mazda_v3.json')